In [ ]:
class Softswitch(object):
    
    
    def __init__(self, serverVSC=None, serverSQL=None,
                       portSQL=None, userVSC=None,
                       userSQL=None, passwordVSC=None,
                       passwordSQL=None, DB_SQL=None):
        
        self.serverVSC = serverVSC
        self.serverSQL = serverSQL
        self.portSQL = portSQL
        self.userVSC = userVSC
        self.userSQL = userSQL
        self.passwordVSC = passwordVSC
        self.passwordSQL = passwordSQL
        self.DB_SQL = DB_SQL
        

In [ ]:
import pyodbc, os

        
class Manager_SQL_Server(Softswitch):
    
    
    def __init__(self, serverSQL, portSQL, userSQL, passwordSQL, DB_SQL):
        super(Manager_SQL_Server, self).__init__(
                                            serverSQL=serverSQL,
                                            portSQL=portSQL,
                                            userSQL=userSQL,
                                            passwordSQL=passwordSQL,
                                            DB_SQL=DB_SQL,
                                            )

        
    def connection_sql(self):
        """ Metodo conexão Database SQL Server """
        self.conection = pyodbc.connect(
                                        'DRIVER={SQL Server};SERVER='+ self.serverSQL +','+ self.portSQL +
                                        ';DATABASE='+ self.DB_SQL +
                                        ';UID='+ self.userSQL +';PWD='+ self.passwordSQL
                                       )
        return self.conection.cursor()
         
    def query_sql(self, command):
        """ Metodo query / read Database SQL Server"""
        self.sql = self.connection_sql()
        return list(self.sql.execute(""" {} """.format(command)))
    
    def commit_sql(self, command):
        """ Metodo commit ( create / update ) Database SQL Server"""
        self.sql = self.connection_sql()
        self.sql.execute( """ {} """.format(command))
        self.sql.commit()
        return True
    

class Macro_SQL:
    # Definir como chegará os valores das q_variable para realizar tratamento
    
    def __init__(self, command, servers):
        
        #self.q_variable = q_variable # Criar macro para tratamento das variaveis → quebrar em List ou string
            # → 
        self.command = command
        self.servers = servers
    
    def macro_sql_query(self):
        """ Macro query consulta em mais de um DB """
        result_query = {}
        if len(self.servers) > 0:
            for server in self.servers:
                connection = Manager_SQL_Server(
                                            serverSQL=self.servers[server]['serverSQL'],
                                            portSQL=self.servers[server]['portSQL'],
                                            userSQL=self.servers[server]['userSQL'],
                                            passwordSQL=self.servers[server]['passwordSQL'],
                                            DB_SQL=self.servers[server]['DB_SQL'],
                                        )
                result_query[server] = connection.query_sql(self.command)
                
            return result_query
        else: return None
        
    def macro_sql_commit(self):
        """ Macro Commit em mais de um DB """
        result_query = {}
        if len(self.servers) > 0:
            for server in self.servers:
                connection = Manager_SQL_Server(
                                            serverSQL=self.servers[server]['serverSQL'],
                                            portSQL=self.servers[server]['portSQL'],
                                            userSQL=self.servers[server]['userSQL'],
                                            passwordSQL=self.servers[server]['passwordSQL'],
                                            DB_SQL=self.servers[server]['DB_SQL'],
                                        )
                result_query[server] = connection.commit_sql(self.command)
                
            return result_query
        else: return None


In [ ]:
dict_db = {
            'name_db': {'serverSQL':'ip_host', 'portSQL':'port_sql', 'userSQL':'user_sql', 'passwordSQL':'pass_sql', 'DB_SQL':'name_sql'},
            'name_db_2': {'serverSQL':'ip_host', 'portSQL':'port_sql', 'userSQL':'user_sql', 'passwordSQL':'pass_sql', 'DB_SQL':'name_sql'},
            }

test_macro_sql = Macro_SQL(command = """
                Entre com sua query para consultar varios SQLs Servers com a mesma arquitetura...
                                 """, servers=dict_db)

result_query = test_macro_sql.macro_sql_query()


In [ ]:
import psycopg2


class (Softswitch):
    Manager_Postgres
    
    def __init__(self, serverSQL, portSQL, userSQL, passwordSQL, DB_SQL):
        super(Manager_Postgres, self).__init__(
                                            serverSQL=serverSQL,
                                            portSQL=portSQL,
                                            userSQL=userSQL,
                                            passwordSQL=passwordSQL,
                                            DB_SQL=DB_SQL,
                                            )


    
    def connection_postgres(self):
        """ Metodo conexão Database SQL Postgres"""
        self.connection = psycopg2.connect(
                                            'host='+ self.serverSQL +' '+
                                            'user='+ self.userSQL +' '+
                                            'dbname='+ self.DB_SQL +' '+
                                            'password='+ self.passwordSQL +' '+
                                            'sslmode=disable'
                                            )
        return self.connection.cursor()
    
    def query_postgres(self, command):
        """ Metodo query / read Database SQL Postgres"""
        self.sql = self.connection_postgres()
        try:
            self.sql.execute(' {} '.format(command))
            return list(self.sql.fetchall())
        except (Exception, psycopg2.DatabaseError) as error:
            return 'Error: '.format(error)
        
        finally:
            if self.connection is not None:
                self.connection.close()

    def commit_postgres(self, command):
        """ Metodo commit ( create / update ) Database SQL Postgres"""
        cursor = self.connection_postgres()
        try:
            cursor.execute(''' {} '''.format(command))
            self.connection.commit()
            cursor.close()
        except (Exception, psycopg2.DatabaseError) as error:
            return 'Error: ' + error
        finally:
            if self.connection is not None:
                self.connection.close()
        return True


class Macro_Postgres:
    
    
    def __init__(self, command, servers):
        
        #self.q_variable = q_variable # Criar macro para tratamento das variaveis.
        self.command = command
        self.servers = servers
    
    def macro_postgres_commit(self):
        """ Macro query consulta  """
        result_query = {}
        if len(self.servers) > 0:
            for server in self.servers:
                connection = Manager_Postgres(
                                            serverSQL=self.servers[server]['serverSQL'],
                                            portSQL=self.servers[server]['portSQL'],
                                            userSQL=self.servers[server]['userSQL'],
                                            passwordSQL=self.servers[server]['passwordSQL'],
                                            DB_SQL=self.servers[server]['DB_SQL'],
                                        )
                result_query[server] = connection.commit_postgres(self.command)
                
                
            return result_query
        else: return None
        
        def macro_postgres_query(self):
            """ Macro query consulta """
            result_query = {}
            if len(self.servers) > 0:
                for server in self.servers:
                    connection = Manager_Postgres(
                                                serverSQL=self.servers[server]['serverSQL'],
                                                portSQL=self.servers[server]['portSQL'],
                                                userSQL=self.servers[server]['userSQL'],
                                                passwordSQL=self.servers[server]['passwordSQL'],
                                                DB_SQL=self.servers[server]['DB_SQL'],
                                            )
                    result_query[server] = connection.query_postgres(self.command)


                return result_query
            else: return None

In [ ]:
from paramiko import SSHClient, AutoAddPolicy


class Manager_SSH():

    def __init__(self, gateway, port, username, password):
        self.gateway = gateway
        self.port = port
        self.username = username
        self.password = password
        self.ssh = SSHClient()

    def command_ssh(self, command):
        self.ssh.load_system_host_keys()
        self.ssh.set_missing_host_key_policy(AutoAddPolicy())
        self.conection = self.ssh.connect(self.gateway, self.port, self.username, self.password)
        stdin, stdout, stderr = self.ssh.exec_command("{}".format(command))
        return str(stdout.read())

    
    
test_ssh = Manager_SSH('host_ip','host_port','host_user', 'host_pass')
test = test_ssh.command_ssh("ls /home/")
test = list(map(str, test.split("\\n")))
test = list(map(str, test[0].split("b'"))) + test[1::]
del(test[0]); del(test[-1])
for tests in test:
    print(tests)